#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

object_type

postal_code

street_id

id_region

house_id

# Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [16]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from scipy.spatial.distance import cdist
from sklearn.model_selection import train_test_split
import seaborn as sns
import pandas as pd
import gc
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression as SklearnLR

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [17]:
!pip install folium

DEPRECATION: Loading egg at c:\users\me\appdata\local\programs\python\python311\lib\site-packages\tk_builder-1.1.12-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Загрузим данные из csv файла в датафрейм.

In [18]:
# Данные в нашем csv-файле разделены через ;
df = pd.read_csv('input_data.csv', sep=';')
print(df.head())

         date     price  level  levels  rooms  area  kitchen_area    geo_lat  \
0  2021-01-01   2451300     15      31      1  30.3           0.0  56.780112   
1  2021-01-01   1450000      5       5      1  33.0           6.0  44.608154   
2  2021-01-01  10700000      4      13      3  85.0          12.0  55.540060   
3  2021-01-01   3100000      3       5      3  82.0           9.0  44.608154   
4  2021-01-01   2500000      2       3      1  30.0           9.0  44.738685   

     geo_lon  building_type  object_type  postal_code  street_id  id_region  \
0  60.699355              0            2     620000.0        NaN         66   
1  40.138381              0            0     385000.0        NaN          1   
2  37.725112              3            0     142701.0   242543.0         50   
3  40.138381              0            0     385000.0        NaN          1   
4  37.713668              3            2     353960.0   439378.0         23   

    house_id  
0  1632918.0  
1        NaN  

Отобразим на карте координаты наших построек.

In [19]:
import folium
from IPython.display import display

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Отобразим координаты первых 1000 квартир
map_df = df.head(1000)

for index, row in map_df.iterrows():
    # Проверка на значение NaN
    if not pd.isna(row['geo_lat']) and not pd.isna(row['geo_lon']):
        folium.Marker(
            location=[row['geo_lat'], row['geo_lon']],
        ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [20]:
import numpy as np

# Функция гаверсинуса
def haversine_distance(lat1, lon1, lat2, lon2):
    # Радиус Земли
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

moscow_coords = (55.7558, 37.6176)
spb_coords = (59.9311, 30.3609)

# Проверка: значение функции гаверсинуса не превышает 20
df['is_Moscow'] = (haversine_distance(df['geo_lat'], df['geo_lon'], moscow_coords[0], moscow_coords[1]) <= 20).astype(int)
df['is_Saint_Peterburg'] = (haversine_distance(df['geo_lat'], df['geo_lon'], spb_coords[0], spb_coords[1]) <= 20).astype(int)

df[['geo_lat', 'geo_lon', 'is_Moscow', 'is_Saint_Peterburg']].head(20)


,geo_lat,geo_lon,is_Moscow,is_Saint_Peterburg
0,56.780112,60.699355,0,0
1,44.608154,40.138381,0,0
2,55.540060,37.725112,0,0
3,44.608154,40.138381,0,0
4,44.738685,37.713668,0,0
5,48.511172,44.566846,0,0
6,55.009914,82.934859,0,0
7,51.834703,107.600571,0,0
8,45.003869,39.086511,0,0
9,53.164362,45.033956,0,0


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [21]:
df = df.drop(columns=[
    'geo_lat', 
    'geo_lon', 
    'object_type', 
    'postal_code', 
    'street_id', 
    'id_region', 
    'house_id'
])

# Просмотрим оставшиеся колонки
print(df.columns.tolist())

['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'building_type', 'is_Moscow', 'is_Saint_Peterburg']


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: 'building_type', 'is_Moscow', 'is_Saint_Peterburg'

Числовые: 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area'

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [22]:
for col in df.columns:
    # Рассмотрим тип данных...
    dtype = df[col].dtype
    # ...И количество уникальных значений в каждой колонке
    unique_count = df[col].nunique()
        
    print(f"\n{col}:")
    print(f"  Тип: {dtype}")
    print(f"  Уникальных значений: {unique_count}")


date:
  Тип: object
  Уникальных значений: 365

price:
  Тип: int64
  Уникальных значений: 636900

level:
  Тип: int64
  Уникальных значений: 51

levels:
  Тип: int64
  Уникальных значений: 51

rooms:
  Тип: int64
  Уникальных значений: 10

area:
  Тип: float64
  Уникальных значений: 15767

kitchen_area:
  Тип: float64
  Уникальных значений: 4223

building_type:
  Тип: int64
  Уникальных значений: 7

is_Moscow:
  Тип: int32
  Уникальных значений: 2

is_Saint_Peterburg:
  Тип: int32
  Уникальных значений: 2


Очевидно, что признаки 'is_Moscow' и 'is_Saint_Peterburg' принимают лишь значения 0 и 1, поэтому автоматически относятся к категориальным. Аналогично можно сказать и про 'building_type': лишь 7 значений, перечисленных текстом в самом начале задания.

In [23]:
categorical_features = ['building_type', 'is_Moscow', 'is_Saint_Peterburg']

# Создаем объект OneHotEncoder с результатом в разреженном формате матрицы
encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
# Обучение и трансформация
encoded_sparse = encoder.fit_transform(df[categorical_features])
# Получение имен новых признаков
feature_names = encoder.get_feature_names_out(categorical_features)

df_encoded = pd.DataFrame.sparse.from_spmatrix(
    encoded_sparse, 
    columns=feature_names,
    index=df.index
)

# Удаление исходных колонок
df = df.drop(columns=categorical_features)
df = pd.concat([df, df_encoded], axis=1)
print(df.columns.tolist())

['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'building_type_0', 'building_type_1', 'building_type_2', 'building_type_3', 'building_type_4', 'building_type_5', 'building_type_6', 'is_Moscow_0', 'is_Moscow_1', 'is_Saint_Peterburg_0', 'is_Saint_Peterburg_1']


**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



Рассмотрим все числовые признаки на крайние (вырожденные) значения.

In [24]:
numeric_features = ['price', 'level', 'levels', 'rooms', 'area', 'kitchen_area']
for feature in numeric_features:
    print(feature, min(df[feature]), max(df[feature]))

price 0 635552400000
level 0 50
levels 0 50
rooms -1 9
area 1.0 499.9
kitchen_area -100.0 408.0


Оставим только строки, соответствующие здравому смыслу: площадь кухни положительна, цена лежит в пределах от 100 тыс. до 1 млрд., этаж квартиры не превышает количество этажей в доме (которых больше 0 штук).

In [25]:
df = df[df['kitchen_area'] > 0]
df = df[(df['price'] > 100000) & (df['price'] < 1000000000)]
df = df[df['level'] <= df['levels']]
df = df[df['levels'] > 0]

In [26]:
# Создадим признаки 'days_since_first' и 'level_ratio'
df['date'] = pd.to_datetime(df['date'])
first_date = df['date'].min()
df['days_since_first'] = (df['date'] - first_date).dt.days
df['level_ratio'] = df['level'] / df['levels']
# Удаляем колонку 'date'
df = df.drop(columns=['date'])

# Описание числовых признаков до нормализации
print(f"\nДо нормализации:")
print(df[numeric_features].describe())

# Нормализация числовых признаков
scaler = StandardScaler()
df[numeric_features] = scaler.fit_transform(df[numeric_features])

# Описание числовых признаков после нормализации
print(f"\nПосле нормализации:")
print(df[numeric_features].describe())
print(f"\nФорма датафрейма: {df.shape}")
print(f"\nКолонки: {df.columns.tolist()}")
print(df.head())


До нормализации:
              price         level        levels         rooms          area  \
count  7.345099e+06  7.345099e+06  7.345099e+06  7.345099e+06  7.345099e+06   
mean   6.440545e+06  6.273402e+00  1.142761e+01  1.836578e+00  5.427448e+01   
std    1.157386e+07  5.184750e+00  7.063698e+00  1.045597e+00  2.608126e+01   
min    1.000500e+05  0.000000e+00  1.000000e+00 -1.000000e+00  1.000000e+00   
25%    2.700000e+06  2.000000e+00  5.000000e+00  1.000000e+00  3.800000e+01   
50%    4.100000e+06  5.000000e+00  9.000000e+00  2.000000e+00  4.800000e+01   
75%    6.900000e+06  9.000000e+00  1.600000e+01  3.000000e+00  6.400000e+01   
max    9.999999e+08  5.000000e+01  5.000000e+01  9.000000e+00  4.990000e+02   

       kitchen_area  
count  7.345099e+06  
mean   1.072775e+01  
std    5.829036e+00  
min    1.000000e-02  
25%    7.000000e+00  
50%    9.200000e+00  
75%    1.260000e+01  
max    4.080000e+02  

После нормализации:
              price         level        levels    

**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [27]:
class KNNRegressorMemoryEfficient:
    def __init__(self, n_neighbors=5, metric='euclidean', max_train_size=100000):
        self.n_neighbors = n_neighbors # кол-во ближайших соседей для усреднения 
        self.metric = metric # метрика расстояния (евклидово расстояние)
        self.max_train_size = max_train_size # максимальный размер обучающей выборки (для экономии памяти)
        self.X_train = None
        self.y_train = None

    # Обучение модели
    def fit(self, X, y):
        X_array = np.array(X, dtype=np.float32)
        y_array = np.array(y, dtype=np.float32)

        # Если данных больше, чем max_train_size, выполняется случайное сэмплирование
        if len(X_array) > self.max_train_size:
            print(f"Сэмплирование: {len(X_array)} -> {self.max_train_size} объектов")
            indices = np.random.choice(len(X_array), self.max_train_size, replace=False)
            self.X_train = X_array[indices]
            self.y_train = y_array[indices]
        else:
            self.X_train = X_array
            self.y_train = y_array

        # Запуск сборщик мусора Python для автоматического освобождения памяти
        del X_array, y_array
        gc.collect()

        return self
    
    # Предсказание значения
    def predict(self, X, batch_size=100, train_batch_size=50000, verbose=True):
        if self.X_train is None or self.y_train is None:
            raise ValueError("Модель не обучена!")

        X_test = np.array(X, dtype=np.float32)
        predictions = np.zeros(len(X_test), dtype=np.float32)
        n_batches = (len(X_test) + batch_size - 1) // batch_size

        if verbose:
            print(f"Предсказание для {len(X_test)} объектов...")

        # Произведение обработки батчами
        for batch_idx, start in enumerate(range(0, len(X_test), batch_size)):
            end = min(start + batch_size, len(X_test))
            test_batch = X_test[start:end]

            if verbose and (batch_idx + 1) % max(1, n_batches // 20) == 0:
                print(f"  Батч {batch_idx + 1}/{n_batches} ({100 * (batch_idx + 1) / n_batches:.1f}%)")

            # Собственно предсказание для текущего батча
            batch_predictions = self._predict_batch(test_batch, train_batch_size)
            predictions[start:end] = batch_predictions

            # Очистка памяти каждые 100 батчей
            if batch_idx % 100 == 0:
                gc.collect()

        if verbose:
            print("Готово!")

        return predictions

    # Реализация предсказания для одного батча
    def _predict_batch(self, test_batch, train_batch_size):
        n_test = len(test_batch)
        # Матрица для хранения k наименьших расстояний 
        all_distances = np.full((n_test, self.n_neighbors), np.inf, dtype=np.float32)
        # Матрица для хранения индексов k ближайших соседей
        all_indices = np.zeros((n_test, self.n_neighbors), dtype=np.int32)

        for train_start in range(0, len(self.X_train), train_batch_size):
            train_end = min(train_start + train_batch_size, len(self.X_train))
            train_batch = self.X_train[train_start:train_end]

            # Непосредственное вычисление расстояний
            distances = cdist(test_batch, train_batch, metric=self.metric).astype(np.float32)

            for i in range(n_test):
                # Объединение текущих k лучших расстояний с новыми расстояниями из текущего батча
                combined_distances = np.concatenate([all_distances[i], distances[i]])
                # Объединение соответствующих индексов
                combined_indices = np.concatenate([
                    all_indices[i],
                    np.arange(train_start, train_end, dtype=np.int32)
                ])

                # Выбор k наименьших расстояний
                top_k_mask = np.argpartition(combined_distances, self.n_neighbors)[:self.n_neighbors]
                all_distances[i] = combined_distances[top_k_mask]
                all_indices[i] = combined_indices[top_k_mask]

            del distances, train_batch

        predictions = np.zeros(n_test, dtype=np.float32)
        # Усреднение k ближайших значений - искомое предсказание
        for i in range(n_test):
            nearest_prices = self.y_train[all_indices[i]]
            predictions[i] = np.mean(nearest_prices)

        return predictions

**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [28]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8,
                 decay_rate=0.9, max_iter=1000, batch_size=32, verbose=True):
        self.learning_rate = learning_rate # Скорость обучения (шаг градиентного спуска)
        self.optimization = optimization # Метод оптимизации (SGD, Momentum, AdaGrad)
        self.epsilon = epsilon # Малая константа для численной стабильности в AdaGrad
        self.decay_rate = decay_rate # Коэффициент затухания для Momentum
        self.max_iter = max_iter # Максимальное количество эпох обучения
        self.batch_size = batch_size # Размер мини-батча
        self.verbose = verbose

        self.weights = None # Веса для каждого признака
        self.bias = None # Cвободный член
        self.loss_history = [] # История значений функции потерь по эпохам

        # Для метода Momentum: накопленная скорость изменения весов и bias
        self.velocity_w = None
        self.velocity_b = None

        # Для метода AdaGrad: накопленные квадраты градиентов
        self.cache_w = None
        self.cache_b = None

    # Инициализация весов
    def _initialize_weights(self, n_features):
        self.weights = np.random.randn(n_features) * np.sqrt(2.0 / n_features)
        self.bias = 0.0
        
        # Инициализация скоростей
        if self.optimization == 'Momentum':
            self.velocity_w = np.zeros(n_features)
            self.velocity_b = 0.0

        # Инициализация кэшей для накопления градиентов
        if self.optimization == 'AdaGrad':
            self.cache_w = np.zeros(n_features)
            self.cache_b = 0.0

    # Вычисление функции потерь
    def _compute_loss(self, X, y):
        n = len(y)
        # Матричное умножение признаков на веса и добавление свободного члена
        predictions = X.dot(self.weights) + self.bias
        mse = np.mean((predictions - y) ** 2)
        return mse

    # Вычисление градиентов
    def _compute_gradients(self, X_batch, y_batch):
        n = len(y_batch)
        predictions = X_batch.dot(self.weights) + self.bias
        # разница между предсказаниями и истинными значениями
        errors = predictions - y_batch

        grad_w = (2.0 / n) * X_batch.T.dot(errors) # Градиент по весам
        grad_b = (2.0 / n) * np.sum(errors) # Градиент по свободному члену

        return grad_w, grad_b

    # Стохастический градиентный спуск
    def _update_weights_sgd(self, grad_w, grad_b):
        # Движение в направлении, противоположном градиенту (минимизация функции потерь)
        self.weights -= self.learning_rate * grad_w
        self.bias -= self.learning_rate * grad_b

    # Оптимизация Momentum
    def _update_weights_momentum(self, grad_w, grad_b):
        # Обновление скорости
        self.velocity_w = self.decay_rate * self.velocity_w - self.learning_rate * grad_w
        self.velocity_b = self.decay_rate * self.velocity_b - self.learning_rate * grad_b

        self.weights += self.velocity_w
        self.bias += self.velocity_b

    # Оптимизация AdaGrad
    def _update_weights_adagrad(self, grad_w, grad_b):
        # Накопление квадратов градиентов
        self.cache_w += grad_w ** 2
        self.cache_b += grad_b ** 2

        self.weights -= self.learning_rate * grad_w / (np.sqrt(self.cache_w) + self.epsilon)
        self.bias -= self.learning_rate * grad_b / (np.sqrt(self.cache_b) + self.epsilon)

    # Обучение модели
    def fit(self, X, y):
        X = np.array(X, dtype=np.float64)
        y = np.array(y, dtype=np.float64)

        n_samples, n_features = X.shape

        self._initialize_weights(n_features)

        if self.verbose:
            print(f"Обучение линейной регрессии методом {self.optimization}")
            print(f"Количество объектов: {n_samples}, признаков: {n_features}")
            print(f"Learning rate: {self.learning_rate}, Max iterations: {self.max_iter}")
            print(f"Batch size: {self.batch_size}")
            print("-" * 70)

        # Случайное перемешивание данных
        for epoch in range(self.max_iter):
            indices = np.random.permutation(n_samples)
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            # Обработка мини-батчами
            for i in range(0, n_samples, self.batch_size):
                X_batch = X_shuffled[i:i + self.batch_size]
                y_batch = y_shuffled[i:i + self.batch_size]

                grad_w, grad_b = self._compute_gradients(X_batch, y_batch)

                if self.optimization == 'SGD':
                    self._update_weights_sgd(grad_w, grad_b)
                elif self.optimization == 'Momentum':
                    self._update_weights_momentum(grad_w, grad_b)
                elif self.optimization == 'AdaGrad':
                    self._update_weights_adagrad(grad_w, grad_b)
                else:
                    raise ValueError(f"Неизвестный метод оптимизации: {self.optimization}")

            # После каждой эпохи: вычисление функции потерь на всем датасете и сохранение в историю
            loss = self._compute_loss(X, y)
            self.loss_history.append(loss)

            if self.verbose and (epoch + 1) % max(1, self.max_iter // 10) == 0:
                print(f"Epoch {epoch + 1}/{self.max_iter}: Loss = {loss:,.4f}")

        if self.verbose:
            print("-" * 70)
            print(f"Обучение завершено! Финальный Loss: {self.loss_history[-1]:,.4f}")

        return self

    # # Предсказание значений
    def predict(self, X):
        if self.weights is None or self.bias is None:
            raise ValueError("Модель не обучена! Сначала вызовите fit()")

        X = np.array(X, dtype=np.float64)
        # Применение линейной модели
        predictions = X.dot(self.weights) + self.bias
        return predictions

# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

Сначала проведём работу с KNN-моделью...

In [29]:
print("Подготовка данных...")
# Разделение на признаки и целевую переменную
X = df.drop(columns=['price'])
y = df['price']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X[:100000], y[:100000], test_size=0.2, random_state=42
)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

# Обучение нашей KNN-модели
knn = KNNRegressorMemoryEfficient(
    n_neighbors=5,
    metric='euclidean',
    max_train_size=100000
)

knn.fit(X_train, y_train)

y_pred = knn.predict(
    X_test,
    batch_size=50,
    train_batch_size=100000,
    verbose=True
)

y_test_arr = np.array(y_test)
y_pred_arr = np.array(y_pred)

# Вычисление ошибок
MSE = np.mean((y_test_arr - y_pred_arr)**2)
RMSE = np.sqrt(MSE)
MAE = np.mean(np.abs(y_test_arr - y_pred_arr))

print(f"MSE: {MSE:,.10f}")
print(f"RMSE: {RMSE:,.10f}")
print(f"MAE: {MAE:,.10f}")


##############################
print("Sklearn KNN")
# Обучение библиотечной KNN-модели
sklearn_knn_regressor = KNeighborsRegressor(n_neighbors=5)
sklearn_knn_regressor.fit(X_train, y_train)
predictions = sklearn_knn_regressor.predict(X_test)

y_test_arr_Sk = np.array(y_test)
y_pred_arr_Sk = np.array(predictions)

# Вычисление ошибок
MSE = np.mean((y_test_arr_Sk - y_pred_arr_Sk)**2)
RMSE = np.sqrt(MSE)
MAE = np.mean(np.abs(y_test_arr_Sk - y_pred_arr_Sk))

print(f"MSE: {MSE:,.10f}")
print(f"RMSE: {RMSE:,.10f}")
print(f"MAE: {MAE:,.10f}")

Подготовка данных...
Train: 80000, Test: 20000
Предсказание для 20000 объектов...
  Батч 20/400 (5.0%)
  Батч 40/400 (10.0%)
  Батч 60/400 (15.0%)
  Батч 80/400 (20.0%)
  Батч 100/400 (25.0%)
  Батч 120/400 (30.0%)
  Батч 140/400 (35.0%)
  Батч 160/400 (40.0%)
  Батч 180/400 (45.0%)
  Батч 200/400 (50.0%)
  Батч 220/400 (55.0%)
  Батч 240/400 (60.0%)
  Батч 260/400 (65.0%)
  Батч 280/400 (70.0%)
  Батч 300/400 (75.0%)
  Батч 320/400 (80.0%)
  Батч 340/400 (85.0%)
  Батч 360/400 (90.0%)
  Батч 380/400 (95.0%)
  Батч 400/400 (100.0%)
Готово!
MSE: 0.2172798265
RMSE: 0.4661328421
MAE: 0.1655287814
Sklearn KNN


c:\Users\me\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\me\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


MSE: 0.2172822984
RMSE: 0.4661354936
MAE: 0.1655703344


...А теперь - с LR-моделью.

In [30]:
print("Подготовка данных...")
# Разделение на признаки и целевую переменную
X = df.drop(columns=['price'])
y = df['price']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X[:100000], y[:100000], test_size=0.2, random_state=42
)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Варианты нашей LR-модели
optimizers = ['SGD', 'Momentum', 'AdaGrad']

# Для каждого варианта - обучение модели и вычисление ошибок
for optimizer in optimizers:
    print("\n" + "="*70)
    print(f"ТЕСТИРОВАНИЕ: {optimizer}")
    print("="*70)

    lr = LinearRegression(
        learning_rate=0.001,
        optimization=optimizer,
        epsilon=1e-8,
        decay_rate=0.9,
        max_iter=100,
        batch_size=1024,
        verbose=True
    )

    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_test)

    MSE = np.mean((y_test - y_pred)**2)
    RMSE = np.sqrt(MSE)
    MAE = np.mean(np.abs(y_test - y_pred))

    print(optimizer)
    print(f"MSE: {MSE:,.10f}")
    print(f"RMSE: {RMSE:,.10f}")
    print(f"MAE: {MAE:,.10f}")

#########################

print("\n" + "="*70)
print("СРАВНЕНИЕ С SKLEARN LinearRegression")
print("="*70)

# Обучение библиотечной LR-модели

sklearn_lr = SklearnLR()
sklearn_lr.fit(X_train, y_train)
y_pred_sklearn = sklearn_lr.predict(X_test)

# Вычисление ошибок
MSE = np.mean((y_test - y_pred_sklearn)**2)
RMSE = np.sqrt(MSE)
MAE = np.mean(np.abs(y_test - y_pred_sklearn))

print(f"\nSklearn LinearRegression:")
print(f"MSE: {MSE:,.10f}")
print(f"RMSE: {RMSE:,.10f}")
print(f"MAE: {MAE:,.10f}")

Подготовка данных...

ТЕСТИРОВАНИЕ: SGD
Обучение линейной регрессии методом SGD
Количество объектов: 80000, признаков: 18
Learning rate: 0.001, Max iterations: 100
Batch size: 1024
----------------------------------------------------------------------
Epoch 10/100: Loss = 0.6961
Epoch 20/100: Loss = 0.6104
Epoch 30/100: Loss = 0.5722
Epoch 40/100: Loss = 0.5536
Epoch 50/100: Loss = 0.5436
Epoch 60/100: Loss = 0.5378
Epoch 70/100: Loss = 0.5342
Epoch 80/100: Loss = 0.5316
Epoch 90/100: Loss = 0.5297
Epoch 100/100: Loss = 0.5284
----------------------------------------------------------------------
Обучение завершено! Финальный Loss: 0.5284
SGD
MSE: 0.3347366675
RMSE: 0.5785643158
MAE: 0.2726950277

ТЕСТИРОВАНИЕ: Momentum
Обучение линейной регрессии методом Momentum
Количество объектов: 80000, признаков: 18
Learning rate: 0.001, Max iterations: 100
Batch size: 1024
----------------------------------------------------------------------
Epoch 10/100: Loss = 0.5357
Epoch 20/100: Loss = 0.53

c:\Users\me\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\me\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
